# adversarial_mnist
Extra required packages:
- matplotlib
- pandas

In [ ]:
import sys

import gurobipy as gp
import numpy as np
import pandas as pd
from joblib import load
from matplotlib import pyplot as plt

from gurobi_ml import add_predictor_constr

In [ ]:
# Load the trained network and the examples
pipe = load("MNIST_50_50.joblib")
X = load("MNIST_first100.joblib")

# Choose an example
exampleno = 26
example = X.iloc[exampleno : exampleno + 1, :]
ex_prob = pipe.predict_proba(example)
output_shape = ex_prob.shape

In [ ]:
pixels = example.to_numpy().reshape((28, 28))
plt.imshow(pixels, cmap="gray")
plt.show()

In [ ]:
pipe.predict(example)

In [ ]:
sortedidx = np.argsort(ex_prob)[0]

In [ ]:
m = gp.Model()
epsilon = 5

x = m.addMVar(example.shape, lb=0.0, ub=1.0, name="X")
absdiff = m.addMVar(example.shape, lb=0, ub=1, name="dplus")
output = m.addMVar(output_shape, lb=-gp.GRB.INFINITY, name="y")

In [ ]:
m.setObjective(output[0, sortedidx[-2]] - output[0, sortedidx[-1]], gp.GRB.MAXIMIZE)

# Bound on the distance to example in norm-1
m.addConstr(absdiff[0, :] >= x[0, :] - example.to_numpy()[0, :])
m.addConstr(absdiff[0, :] >= -x[0, :] + example.to_numpy()[0, :])
m.addConstr(absdiff[0, :].sum() <= epsilon)

In [ ]:
# Change last layer activation to identity
pipe.steps[-1][1].out_activation_ = "identity"
# Code to add the neural network to the constraints
ml2grb = add_predictor_constr(m, pipe, x, output)

# Restore activation
pipe.steps[-1][1].out_activation_ = "softmax"

In [ ]:
ml2grb.print_stats()

In [ ]:
m.Params.OutputFlag = 1
m.Params.TimeLimit = 25
m.Params.BestBdStop = 0.0
m.Params.BestObjStop = 0.0
m.optimize()

In [ ]:
pixels = x.X.reshape((28, 28))
plt.imshow(pixels, cmap="gray")
plt.show()

In [ ]:
example_mod = pd.DataFrame(data=x.X, columns=example.columns, index=example.index)
pipe.predict(example_mod)

copyright © 2022 Gurobi Optimization, LLC